# Balanced network of excitatory and inhibitory neurons.

An implementation of benchmarks from Brette et al. (2007) Journal of Computational Neuroscience 23: 349-398
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2638500/ (see Fig. 24). 

In the paper, they compared simulations from NEST and NEURON. Here, we are going to simulate the balanced network with SpiNNaker.

The network is based on the COBA models of Vogels & Abbott (J. Neurosci, 2005) https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6725859/ .

You can look also the explicit model in the supplementary material of Brette et al. (2007) |https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2638500/#APP2

# modules

In [1]:
import os
import time
import datetime
import pickle
import warnings
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from scipy.signal import welch
from fooof import FOOOF
import os
import time
import datetime
import pickle
import warnings
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from scipy.signal import welch
from fooof import FOOOF
from pyNN import space
from pyNN.random import NumpyRNG, RandomDistribution
import python_utils as pu
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib.pyplot as plt
from fooof import FOOOF
from pyNN import space
import numpy as np
import matplotlib.pyplot as plt
import pickle
import time
import datetime
import json
import warnings
from fooof import FOOOF
from scipy.signal import welch
from matplotlib import colors
import os

/tmp/ipykernel_65467/2961850921.py:10: DeprecationWarning: 
The `fooof` package is being deprecated and replaced by the `specparam` (spectral parameterization) package.
This version of `fooof` (1.1) is fully functional, but will not be further updated.
New projects are recommended to update to using `specparam` (see Changelog for details).
  from fooof import FOOOF


# routines

In [2]:
def computeVAnetwork(Gexc=4.0, Ginh=51.0, AF=1, RATIO=4, DEGENERATION=True, COMPENSATION=True, RESERVE_ORDER=1, NCELL=120, PCONN=0.02, THALCELL=20, TSIM=1000, RATE=4, SHOT=False, plot=True, savePath='../outputs/', fileName='VA_balance-network_sweep'):

    warnings.filterwarnings('ignore')
    os.makedirs(savePath, exist_ok=True)

     # PyNN backend: preferisce SpiNNaker, fallback su Brian2
    try:
        import pyNN.spiNNaker as sim
    except ModuleNotFoundError:
        import pyNN.brian2 as sim

    # Utilità personalizzate
    import python_utils as pu

    # Disattiva warning inutili
    warnings.filterwarnings('ignore')

    # Parametri di salvataggio
    fileName = 'VA_balance-network-sweep'
    savePath = '../outputs/'  # Assicurati che esista o usa os.makedirs()
    dt_string = datetime.datetime.today().strftime('%Y%m%d_%H%M%S')  # formato sicuro per filesystem
    tag = dt_string
    saveName = f'{savePath}{fileName}-{tag}'
    os.makedirs(saveName, exist_ok=True)

    print(f'Saving in: {saveName}')

    # Dizionario per i parametri
    PARS = {}

    dt = 1
    tstop = TSIM
    delay = 2

    sim.setup(timestep=dt, min_delay=delay, max_delay=delay)
    rng = NumpyRNG(seed=98766987, parallel_safe=True)

    celltype = sim.IF_cond_exp

    area = 20000. * 1e-8
    tau_m = 20.
    cm = 1. * area * 1000
    g_leak = 5e-5
    Rm = 1e-6 / (g_leak * area)
    assert tau_m == cm * Rm

    cell_params = {
        'tau_m': tau_m,
        'tau_syn_E': 5.,
        'tau_syn_I': 10.,
        'v_rest': -60.,
        'v_reset': -60.,
        'v_thresh': -50.,
        'cm': cm,
        'tau_refrac': 5.,
        'i_offset': 0,
        'e_rev_E': 0.,
        'e_rev_I': -80.
    }

    NCELL = NCELL*AF if DEGENERATION==True else NCELL
    n = int(round(NCELL)) #1500*AF
    r_ei = RATIO #int(round(4.0*(1+(1-AF))))
    n_exc = int(round((n * r_ei / (1 + r_ei))))
    n_inh = n - n_exc

    pops = {
        'exc': sim.Population(n_exc, celltype(**cell_params), label="Excitatory_Cells"),
        'inh': sim.Population(n_inh, celltype(**cell_params), label="Inhibitory_Cells")
    }

    for pop in ['exc', 'inh']:
        pops[pop].record(["spikes", 'v', 'gsyn_exc', 'gsyn_inh'])
        pops[pop].initialize(v=RandomDistribution('uniform', [-60., -50.], rng=rng))

    # compensation
    """
    w_exc = Gexc * 1e-3 * (1+(1-AF) ** ORDERCOMP) if COMPENSATION==True else Gexc * 1e-3
    w_inh = Ginh * 1e-3 * (1+(1-AF) ** ORDERCOMP) if COMPENSATION==True else Ginh * 1e-3
    pconn = PCONN*(1+(1-AF)** ORDERCOMP) if COMPENSATION==True else PCONN
    """
    # compensation basata su modello sigmoide della riserva cognitiva
    #def sigmoid_reserve(AF, theta, alpha):
    #    return 1 / (1 + np.exp(alpha * (AF - theta)))
    #ALPHA, THETA = RESERVE_EFFECT
    #sigmoid_term = sigmoid_reserve(AF, theta=THETA, alpha=ALPHA)
    compensation = 1+(1-AF)**RESERVE_ORDER if COMPENSATION else 1
    w_exc = Gexc * 1e-3 * compensation
    w_inh = Ginh * 1e-3 *  compensation
    pconn = PCONN * compensation

    exc_synapses = sim.StaticSynapse(weight=w_exc, delay=delay)
    inh_synapses = sim.StaticSynapse(weight=w_inh, delay=delay)
    exc_conn = sim.FixedProbabilityConnector(pconn, rng=rng)
    inh_conn = sim.FixedProbabilityConnector(pconn, rng=rng)

    connections = {
        'e2e': sim.Projection(pops['exc'], pops['exc'], exc_conn, receptor_type='excitatory', synapse_type=exc_synapses),
        'e2i': sim.Projection(pops['exc'], pops['inh'], exc_conn, receptor_type='excitatory', synapse_type=exc_synapses),
        'i2e': sim.Projection(pops['inh'], pops['exc'], inh_conn, receptor_type='inhibitory', synapse_type=inh_synapses),
        'i2i': sim.Projection(pops['inh'], pops['inh'], inh_conn, receptor_type='inhibitory', synapse_type=inh_synapses)
    }

    #pops['thalamus'] = sim.Population(20, sim.SpikeSourcePoisson(rate=100, duration=50), label="expoisson") #duration=50
    pops['thalamus'] = sim.Population(THALCELL, sim.SpikeSourcePoisson(rate=RATE, duration=tstop if SHOT==False else 50), label="expoisson") #duration=50
    pops['thalamus'].record("spikes")

    talCX_conn = sim.FixedProbabilityConnector(0.01) #sim.OneToOneConnector()
    connections['ext2e'] = sim.Projection(pops['thalamus'], pops['exc'], talCX_conn, receptor_type='excitatory', synapse_type=sim.StaticSynapse(weight=0.1))
    connections['ext2i'] = sim.Projection(pops['thalamus'], pops['inh'], talCX_conn, receptor_type='excitatory', synapse_type=sim.StaticSynapse(weight=0.1))

    tic = time.time()
    sim.run(tstop)
    toc = time.time() - tic
    print(f"Simulation time: {toc:.2f} s")

    dt_string = datetime.datetime.today().isoformat()
    saveName = f'{savePath}VA_balance-network_sweep-{dt_string}'

    stateVars = {}
    for pop in pops:
        for rec in ['v', 'gsyn_inh', 'gsyn_exc', 'spikes']:
            pops[pop].write_data(f'{saveName}-{rec}.pkl')
        stateVars[pop] = pops[pop].get_data()

    results = pu.recover_results(stateVars, do_print=False)
    fooof_pars = {}

    fs = dt * 1e3
    nperseg = 128

    for idx, (label, color) in enumerate(zip(['inh', 'exc'], ['b', 'r'])):
        feat = 'v'
        sig = np.asarray(results[label, feat]).T
        f, psd = welch(sig, fs=fs, axis=0, nperseg=nperseg)
        psd_mean = np.mean(psd, axis=1)
        psd_sem = np.std(psd, axis=1) / np.sqrt(psd.shape[1])

        fm = FOOOF(verbose=False)
        fm.fit(f, psd_mean)
        fooof_pars[label] = {
            'aperiodic_params': fm.get_params('aperiodic_params'),
            'peak_params': fm.get_params('peak_params')
        }

        #if plot: fm.report(f, psd_mean)   
    
    results['fooof_pars'] = fooof_pars

    spike_counts_exc = pops['exc'].get_spike_counts()
    spike_counts_inh = pops['inh'].get_spike_counts()

    rate_exc = np.mean(list(spike_counts_exc.values())) / (tstop / 1000)
    rate_inh = np.mean(list(spike_counts_inh.values())) / (tstop / 1000)
    rate_total = np.mean([rate_exc, rate_inh]) 

    results['mean_rate_exc'] = rate_exc
    results['mean_rate_inh'] = rate_inh
    results['mean_rate_total'] = rate_total

    total_exc_weight = 0.0
    total_inh_weight = 0.0
    for key in ['e2e', 'e2i']:
        total_exc_weight += np.sum(connections[key].get(['weight'], format='list'))
    for key in ['i2e', 'i2i']:
        total_inh_weight += np.sum(connections[key].get(['weight'], format='list'))

    structural_balance_ratio = total_exc_weight / (total_exc_weight + total_inh_weight)

    results['structural_balance'] = {
        'total_exc_weight': total_exc_weight,
        'total_inh_weight': total_inh_weight,
        'exc_ratio': structural_balance_ratio,
        #'pconn': pconn,
        #'w_exc': w_exc,
        #'w_inh': w_inh
    }

    
    func_balance = {}
    for pop in ['exc', 'inh']:
        g_exc = np.asarray(results[pop, 'gsyn_exc'])
        g_inh = np.asarray(results[pop, 'gsyn_inh'])
        g_total = g_exc + g_inh
        ratio = np.mean(g_exc / (g_total + 1e-12))
        func_balance[pop] = ratio

    func_balance['overall'] = np.mean([func_balance['exc'], func_balance['inh']])
    results['functional_balance'] = func_balance
    
    #topometrics = pu.extract_topological_metrics(connections, n_exc, n_inh)
    #results['topological_metrics'] = topometrics
    #print(results['topological_metrics'])
    print(results['functional_balance'])
    print(results['structural_balance'] )
    print(results['mean_rate_exc'])
    print(results['mean_rate_inh'])
    print(results['mean_rate_total'])
    

    if plot:
        pu.plot_population_with_fooof(results, 'exc', saveDir='./outputs', fileName='VA_balance-network_sweep',rate=RATE)
        pu.plot_population_with_fooof(results, 'inh', saveDir='./outputs', fileName='VA_balance-network_sweep',rate=RATE)


    return results


# procs

In [3]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

RATIO = 4
Gexc = RATIO
Ginh = 13 * RATIO
AF_values = np.linspace(1, 0.7, 15)
NCELL = 115
THALCELL = 20
TSIM = 1000 * 5
RATE = 4
PCONN = 0.05
DEGENERATION = True
SHOT = False
AF_values = np.linspace(1, 0.7, 15)
RESERVE_ORDERS = [1/3, 1/2, 1, 2, 3]
sweep_data = []

for COMPENSATION in [False, True]:
    for RESERVE_ORDER in RESERVE_ORDERS:
        for AF in tqdm(AF_values, desc=f"COMP={COMPENSATION}, ORDER={RESERVE_ORDER}"):
            results = computeVAnetwork(
                Gexc=Gexc, Ginh=Ginh, AF=AF, RATIO=RATIO, RESERVE_ORDER=RESERVE_ORDER,
                DEGENERATION=DEGENERATION, COMPENSATION=COMPENSATION, 
                PCONN=PCONN, NCELL=NCELL, THALCELL=THALCELL,
                SHOT=SHOT, TSIM=TSIM, RATE=RATE, plot=False
            )
            sweep_data.append({ ... })  # come sopra


COMP=False, ORDER=0.3333333333333333:   0%|          | 0/15 [00:00<?, ?it/s]INFO       Cache size for target 'cython': 2946756011 MB.
You can call clear_cache('cython') to delete all files from the cache or manually delete files in the '/home/bbpnrsoa/.cython/brian_extensions' directory. [brian2]


Saving in: ../outputs/VA_balance-network-sweep-20250603_174323


INFO       No numerical integration method specified for group 'neurongroup_1', using method 'euler' (took 0.02s, trying other methods took 0.08s). [brian2.stateupdaters.base.method_choice]
INFO       No numerical integration method specified for group 'neurongroup', using method 'euler' (took 0.01s, trying other methods took 0.03s). [brian2.stateupdaters.base.method_choice]


Simulation time: 1.70 s


COMP=False, ORDER=0.3333333333333333:   7%|▋         | 1/15 [00:07<01:42,  7.35s/it]

...

gsyn_inh
v
gsyn_exc
gsyn_inh
v
gsyn_exc
{'exc': 0.5201772189308065, 'inh': 0.4633677146789396, 'overall': 0.4917724668048731}
{'total_exc_weight': 17541.15647124927, 'total_inh_weight': 2403.4530678644314, 'exc_ratio': 0.8794936013587542}
5.790909090909091
2.8941176470588235
4.3425133689839575
Saving in: ../outputs/VA_balance-network-sweep-20250603_182844


INFO       No numerical integration method specified for group 'neurongroup_298', using method 'euler' (took 0.01s, trying other methods took 0.03s). [brian2.stateupdaters.base.method_choice]
INFO       No numerical integration method specified for group 'neurongroup_299', using method 'euler' (took 0.01s, trying other methods took 0.03s). [brian2.stateupdaters.base.method_choice]


Simulation time: 19.99 s


COMP=True, ORDER=3: 100%|██████████| 15/15 [06:07<00:00, 24.50s/it]

gsyn_inh
v
gsyn_exc
gsyn_inh
v
gsyn_exc
{'exc': 0.3981335212950961, 'inh': 0.3213944007058812, 'overall': 0.3597639610004887}
{'total_exc_weight': 16475.092728000003, 'total_inh_weight': 1818.883816, 'exc_ratio': 0.9005747158565942}
3.20625
1.1
2.153125


In [6]:
sweep_data

[{Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},
 {Ellipsis},

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, spearmanr

def plot_results_AF_sweep(AF_values, results_data, collapsed=False, savepath="plot_AF_sweep.png"):
    titles = ["Mean Rate", "FOOOF Exponent", "FOOOF Offset", "Structural Balance", "Functional Balance"]
    metric_keys = ["rates", "fooof_exponent", "fooof_offset", "structural_ratio", "functional_ratio"]
    
    colors = {False: "gray", True: "black"}
    labels = {False: "NO COMP", True: "COMP"}

    if collapsed:
        fig, axs = plt.subplots(5, 1, figsize=(10, 18), sharey='row')
    else:
        fig, axs = plt.subplots(5, 2, figsize=(15, 18), sharey='row')

    for i, (title, metric_key) in enumerate(zip(titles, metric_keys)):
        for j, COMP in enumerate([False, True]):
            ax = axs[i] if collapsed else axs[i, j]
            d = results_data[COMP]

            exc_vals = d[f"{metric_key}_exc"]
            inh_vals = d[f"{metric_key}_inh"]
            y_mean = np.mean(np.vstack([exc_vals, inh_vals]), axis=0)

            if collapsed:
                sns.regplot(x=AF_values, y=y_mean, ax=ax, scatter=False, color=colors[COMP], label=labels[COMP], order=1)
                pear, _ = pearsonr(AF_values, y_mean)
                spear, _ = spearmanr(AF_values, y_mean)
                ax.text(0.05, 0.9 - 0.1 * j, f"{labels[COMP]}: r={pear:.2f}, ρ={spear:.2f}",
                        transform=ax.transAxes, fontsize=10, color=colors[COMP])
            else:
                ax.plot(AF_values, exc_vals, label='EXC', marker='o', linewidth=2, color='r', alpha=0.5)
                ax.plot(AF_values, inh_vals, label='INH', marker='o', linewidth=2, color='b', alpha=0.5)
                sns.regplot(x=AF_values, y=y_mean, ax=ax, scatter=False, color='gray', order=1, label='Mean REG')
                pear, _ = pearsonr(AF_values, y_mean)
                spear, _ = spearmanr(AF_values, y_mean)
                ax.set_title(f"{title} – {'COMP' if COMP else 'NO COMP'}\nPearson={pear:.2f}, Spearman={spear:.2f}", fontsize=11)

            if title == "Structural Balance":
                ax.set_yscale('log')
            if i == 4:
                ax.set_xlabel("Atrophy Factor (AF)", fontsize=11)
            if (not collapsed and j == 0) or collapsed:
                ax.set_ylabel(title, fontsize=11)

            ax.grid(True, linestyle='--', alpha=0.5)
            ax.legend()

    plt.tight_layout()
    plt.savefig(savepath, dpi=300)
    plt.show()


In [5]:
plot_results_AF_sweep(AF_values, results_data, collapsed=False, savepath="plot_AF_sweep.png")

NameError: name 'results_data' is not defined

In [ ]:
plot_results_AF_sweep(AF_values, results_data, collapsed=True, savepath="plot_AF_sweep.png")

In [ ]:
stop

In [ ]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

RATIO = 4
Gexc = RATIO
Ginh = 13 * RATIO
AF_values = np.linspace(1, 0.7, 15)
NCELL = 115
THALCELL = 20
TSIM = 1000 * 5
RATE = 4
PCONN = 0.05
DEGENERATION = True
RESERVE_ORDER = 1
SHOT = False

# Dati separati per i due casi
results_data = {
    True: {
        "rates_overall": [],
        "rates_exc": [],
        "rates_inh": [],
        "fooof_offset_exc": [],
        "fooof_offset_inh": [],
        "fooof_exponent_exc": [],
        "fooof_exponent_inh": [],
        "structural_ratio_overall": [],
        "structural_ratio_exc": [],
        "structural_ratio_inh": [],
        "functional_ratio_overall": [],
        "functional_ratio_exc": [],
        "functional_ratio_inh": [],
    },
    False: {
        "rates_overall": [],
        "rates_exc": [],
        "rates_inh": [],
        "fooof_offset_exc": [],
        "fooof_offset_inh": [],
        "fooof_exponent_exc": [],
        "fooof_exponent_inh": [],
        "structural_ratio_overall": [],
        "structural_ratio_exc": [],
        "structural_ratio_inh": [],
        "functional_ratio_overall": [],
        "functional_ratio_exc": [],
        "functional_ratio_inh": [],
    }
}

for COMPENSATION in [True, False]:
    for AF in tqdm(AF_values, desc=f"AF sweep – COMP={COMPENSATION}"):
        for RESERVE_ORDER in [1/3, 1/2, 1, 2, 3]:
        results = computeVAnetwork(
                Gexc=Gexc, Ginh=Ginh, AF=AF, RATIO=RATIO, RESERVE_ORDER=RESERVE_ORDER,
                DEGENERATION=DEGENERATION, COMPENSATION=COMPENSATION, 
                PCONN=PCONN, NCELL=NCELL, THALCELL=THALCELL,
                SHOT=SHOT, TSIM=TSIM, RATE=RATE, plot=False
            )

        d = results_data[COMPENSATION]
        d["rates_overall"].append(results["mean_rate_total"])
        d["rates_exc"].append(results["mean_rate_exc"])
        d["rates_inh"].append(results["mean_rate_inh"])
        d["fooof_offset_inh"].append(results["fooof_pars"]["inh"]["aperiodic_params"][0])
        d["fooof_exponent_inh"].append(results["fooof_pars"]["inh"]["aperiodic_params"][1])
        d["fooof_offset_exc"].append(results["fooof_pars"]["exc"]["aperiodic_params"][0])
        d["fooof_exponent_exc"].append(results["fooof_pars"]["exc"]["aperiodic_params"][1])
        d["structural_ratio_exc"].append(results["structural_balance"]["total_exc_weight"])
        d["structural_ratio_inh"].append(results["structural_balance"]["total_inh_weight"])
        d["structural_ratio_inh"].append(results["structural_balance"]["w_exc"])
        d["structural_ratio_exc"].append(results["structural_balance"]["w_inh"])
        d["structural_ratio_exc"].append(results["structural_balance"]["pconn"])
        d["structural_ratio_inh"].append(results["structural_balance"]["pconn"])
        
        d["structural_ratio_overall"].append(results["structural_balance"]["exc_ratio"])
        d["functional_ratio_overall"].append(results["functional_balance"]["overall"])
        d["functional_ratio_exc"].append(results["functional_balance"]["exc"])
        d["functional_ratio_inh"].append(results["functional_balance"]["inh"])



import seaborn as sns
from scipy.stats import spearmanr, pearsonr

fig, axs = plt.subplots(5, 2, figsize=(15, 18), sharey='row')
titles = ["Mean Rate", "FOOOF Exponent", "FOOOF Offset", "Structural Balance", "Functional Balance"]
metric_keys = ["rates", "fooof_exponent", "fooof_offset", "structural_ratio", "functional_ratio"]

for i, (title, metric_key) in enumerate(zip(titles, metric_keys)):
    for j, COMP in enumerate([False, True]):
        ax = axs[i, j]
        d = results_data[COMP]

        # Tracce exc e inh
        plot_labels = ["exc", "inh"]
        plot_colors = ["r", "b"]
        exc_vals = d[f"{metric_key}_exc"]
        inh_vals = d[f"{metric_key}_inh"]
        ax.plot(AF_values, exc_vals, label='EXC', marker='o', linewidth=2, color='r', alpha=0.5)
        ax.plot(AF_values, inh_vals, label='INH', marker='o', linewidth=2, color='b', alpha=0.5)

        # Retta di regressione sulla media tra EXC e INH
        y_mean = np.mean(np.vstack([exc_vals, inh_vals]), axis=0)
        sns.regplot(x=AF_values, y=y_mean, ax=ax, scatter=False, color='gray', order=1, label='Mean REG')

        # Calcolo correlazioni
        pear, _ = pearsonr(AF_values, y_mean)
        spear, _ = spearmanr(AF_values, y_mean)

        ax.set_title(f"{title} – {'COMP' if COMP else 'NO COMP'}\nPearson={pear:.2f}, Spearman={spear:.2f}", fontsize=11)

        if title == "Structural Balance":
            ax.set_yscale('log')
        if i == 4:
            ax.set_xlabel("Atrophy Factor (AF)", fontsize=11)
        if j == 0:
            ax.set_ylabel(title, fontsize=11)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.legend()

plt.tight_layout()
plt.savefig("sweep_AF_COMPvNOCOMP.png", dpi=300)
plt.show()

import seaborn as sns
from scipy.stats import spearmanr, pearsonr
import matplotlib.pyplot as plt
import numpy as np

fig, axs = plt.subplots(5, 1, figsize=(10, 18), sharey='row')
titles = ["Mean Rate", "FOOOF Exponent", "FOOOF Offset", "Structural Balance", "Functional Balance"]
metric_keys = ["rates", "fooof_exponent", "fooof_offset", "structural_ratio", "functional_ratio"]

colors = {False: "gray", True: "black"}
labels = {False: "NO COMP", True: "COMP"}

for i, (title, metric_key) in enumerate(zip(titles, metric_keys)):
    ax = axs[i]
    for COMP in [False, True]:
        d = results_data[COMP]
        exc_vals = d[f"{metric_key}_exc"]
        inh_vals = d[f"{metric_key}_inh"]
        y_mean = np.mean(np.vstack([exc_vals, inh_vals]), axis=0)
        
        sns.regplot(x=AF_values, y=y_mean, ax=ax, scatter=False, color=colors[COMP], label=labels[COMP], order=1)

        # Correlazioni
        pear, _ = pearsonr(AF_values, y_mean)
        spear, _ = spearmanr(AF_values, y_mean)
        ax.text(0.05, 0.9 - 0.1 * COMP, f"{labels[COMP]}: r={pear:.2f}, ρ={spear:.2f}",
                transform=ax.transAxes, fontsize=10, color=colors[COMP])

    if title == "Structural Balance":
        ax.set_yscale('log')
    ax.set_title(title, fontsize=12)
    if i == 4:
        ax.set_xlabel("Atrophy Factor (AF)", fontsize=12)
    ax.set_ylabel(title, fontsize=11)
    ax.grid(True, linestyle='--', alpha=0.5)
    ax.legend()

plt.tight_layout()
plt.savefig("collapsed_regressions_AF_COMPvNOCOMP.png", dpi=300)
plt.show()


In [ ]:
import seaborn as sns
from scipy.stats import spearmanr, pearsonr

fig, axs = plt.subplots(5, 2, figsize=(15, 18), sharey='row')
titles = ["Mean Rate", "FOOOF Exponent", "FOOOF Offset", "Structural Balance", "Functional Balance"]
metric_keys = ["rates", "fooof_exponent", "fooof_offset", "structural_ratio", "functional_ratio"]

for i, (title, metric_key) in enumerate(zip(titles, metric_keys)):
    for j, COMP in enumerate([False, True]):
        ax = axs[i, j]
        d = results_data[COMP]

        # Tracce exc e inh
        plot_labels = ["exc", "inh"]
        plot_colors = ["r", "b"]
        exc_vals = d[f"{metric_key}_exc"]
        inh_vals = d[f"{metric_key}_inh"]
        ax.plot(AF_values, exc_vals, label='EXC', marker='o', linewidth=2, color='r', alpha=0.5)
        ax.plot(AF_values, inh_vals, label='INH', marker='o', linewidth=2, color='b', alpha=0.5)

        # Retta di regressione sulla media tra EXC e INH
        y_mean = np.mean(np.vstack([exc_vals, inh_vals]), axis=0)
        sns.regplot(x=AF_values, y=y_mean, ax=ax, scatter=False, color='gray', order=1, label='Mean REG')

        # Calcolo correlazioni
        pear, _ = pearsonr(AF_values, y_mean)
        spear, _ = spearmanr(AF_values, y_mean)

        ax.set_title(f"{title} – {'COMP' if COMP else 'NO COMP'}\nPearson={pear:.2f}, Spearman={spear:.2f}", fontsize=11)

        if title == "Structural Balance":
            ax.set_yscale('log')
        if i == 4:
            ax.set_xlabel("Atrophy Factor (AF)", fontsize=11)
        if j == 0:
            ax.set_ylabel(title, fontsize=11)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.legend()

plt.tight_layout()
plt.savefig("sweep_AF_COMPvNOCOMP.png", dpi=300)
plt.show()


In [ ]:
import seaborn as sns
from scipy.stats import spearmanr, pearsonr
import matplotlib.pyplot as plt
import numpy as np

fig, axs = plt.subplots(5, 1, figsize=(10, 18), sharey='row')
titles = ["Mean Rate", "FOOOF Exponent", "FOOOF Offset", "Structural Balance", "Functional Balance"]
metric_keys = ["rates", "fooof_exponent", "fooof_offset", "structural_ratio", "functional_ratio"]

colors = {False: "gray", True: "black"}
labels = {False: "NO COMP", True: "COMP"}

for i, (title, metric_key) in enumerate(zip(titles, metric_keys)):
    ax = axs[i]
    for COMP in [False, True]:
        d = results_data[COMP]
        exc_vals = d[f"{metric_key}_exc"]
        inh_vals = d[f"{metric_key}_inh"]
        y_mean = np.mean(np.vstack([exc_vals, inh_vals]), axis=0)
        
        sns.regplot(x=AF_values, y=y_mean, ax=ax, scatter=False, color=colors[COMP], label=labels[COMP], order=1)

        # Correlazioni
        pear, _ = pearsonr(AF_values, y_mean)
        spear, _ = spearmanr(AF_values, y_mean)
        ax.text(0.05, 0.9 - 0.1 * COMP, f"{labels[COMP]}: r={pear:.2f}, ρ={spear:.2f}",
                transform=ax.transAxes, fontsize=10, color=colors[COMP])

    if title == "Structural Balance":
        ax.set_yscale('log')
    ax.set_title(title, fontsize=12)
    if i == 4:
        ax.set_xlabel("Atrophy Factor (AF)", fontsize=12)
    ax.set_ylabel(title, fontsize=11)
    ax.grid(True, linestyle='--', alpha=0.5)
    ax.legend()

plt.tight_layout()
plt.savefig("collapsed_regressions_AF_COMPvNOCOMP.png", dpi=300)
plt.show()


# extra

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Funzione sigmoide centrata per passare sempre da 1 (AF=0) a 0 (AF=1)
def sigmoid_reserve(AF, alpha=10):
    theta = 0.5  # fisso per garantire simmetria tra 0 e 1
    return 1 / (1 + np.exp(alpha * (AF - theta)))

# Valori di AF decrescenti da 1 a 0 (come la tua richiesta)
AF_values = np.linspace(1, 0, 200)

# Diversi alpha per cambiare solo la pendenza
alpha_values = [3, 5, 10, 20, 40]

# Plot
plt.figure(figsize=(8, 5))
for alpha in alpha_values:
    y = sigmoid_reserve(AF_values, alpha=alpha)
    plt.plot(AF_values, y, label=f'α={alpha}')

plt.title('Sigmoidi con stesso inizio/fine ma diversa pendenza')
plt.xlabel('Atrophy Factor (AF)')
plt.ylabel('Reserve Scaling')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
